In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1tO4KwoShDU3ngnCO2YPoytuUoD8dbcF-'}) 
downloaded.GetContentFile('gun200.zip')

In [ ]:
!unzip 'gun200.zip'

Archive:  gun200.zip
replace gun200/gunaud/0/0.mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import cv2, numpy as np, glob, os,shutil, keras, matplotlib.pyplot as plt,tensorflow as tf,matplotlib.mlab as mplt, sys
from matplotlib import style
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout,ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split as tt
import scipy.io.wavfile
from math import sqrt
import librosa
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve,f1_score,average_precision_score,confusion_matrix
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler as sd
from sklearn.model_selection import StratifiedKFold
import subprocess
from sklearn.decomposition import PCA
from IPython.display import Audio
from keras.models import Model
from keras import optimizers

Using TensorFlow backend.


In [ ]:
#librosa.feature.chroma_stft(audio,sr=22500),librosa.feature.chroma_cqt(audio,sr=22500), librosa.feature.melspectrogram(y=audio,sr=22500)
def feature1(audio):
  return np.concatenate((librosa.feature.mfcc(audio,sr=22500,n_mfcc=44),librosa.feature.tempogram(audio,sr=22500)),axis=0)

def feature2(audio):
  return np.concatenate((librosa.feature.mfcc(audio,sr=22500,n_mfcc=44),librosa.feature.tempogram(audio,sr=22500),librosa.feature.melspectrogram(y=audio,sr=22500)),axis=0)

def feature3(audio):
  m=librosa.feature.mfcc(audio,sr=22500,n_mfcc=44)
  dmfcc=librosa.feature.delta(m)
  ddmfcc=librosa.feature.delta(dmfcc)
  return np.concatenate((m,dmfcc,ddmfcc),axis=0)

def feature4(audio):
  #librosa.feature.chroma_stft(audio,sr=22500),librosa.feature.chroma_cqt(audio,sr=22500), 
  return np.concatenate((librosa.feature.chroma_cens(audio,sr=22500),librosa.feature.mfcc(audio,sr=22500,n_mfcc=44) ,
                         librosa.feature.melspectrogram(y=audio,sr=22500),
               librosa.feature.rms(audio), librosa.feature.rmse(audio), librosa.feature.spectral_centroid(audio,sr=22500),
               librosa.feature.spectral_bandwidth(audio,sr=22500),librosa.feature.spectral_contrast(audio,sr=22500),
               librosa.feature.spectral_flatness(audio),librosa.feature.spectral_rolloff(audio,sr=22500),
               librosa.feature.poly_features(audio,sr=22500),librosa.feature.tonnetz(audio,sr=22500),
               librosa.feature.zero_crossing_rate(audio)),axis=0)

def feature5(audio):
  return np.concatenate((librosa.feature.mfcc(audio,sr=22500,n_mfcc=44),librosa.feature.melspectrogram(y=audio,sr=22500)),axis=0)

  
  
def feature6(audio):
  #librosa.feature.chroma_stft(audio,sr=22500),librosa.feature.chroma_cqt(audio,sr=22500), 
  return np.concatenate((librosa.feature.chroma_cens(audio,sr=22500),librosa.feature.mfcc(audio,sr=22500,n_mfcc=44) ,librosa.feature.melspectrogram(y=audio,sr=22500),
               librosa.feature.rms(audio), librosa.feature.rmse(audio), librosa.feature.spectral_centroid(audio,sr=22500),librosa.feature.tempogram(audio,sr=22500),
               librosa.feature.spectral_bandwidth(audio,sr=22500),librosa.feature.spectral_contrast(audio,sr=22500),librosa.feature.spectral_flatness(audio),librosa.feature.spectral_rolloff(audio,sr=22500),
               librosa.feature.poly_features(audio,sr=22500),librosa.feature.tonnetz(audio,sr=22500),librosa.feature.zero_crossing_rate(audio)),axis=0)

  
def feature7(audio):
  return librosa.feature.mfcc(audio,sr=22500,n_mfcc=44)
  
def feature8(audio):
  return librosa.feature.tempogram(audio,sr=22500)

def feature9(audio):
  return librosa.feature.melspectrogram(y=audio,sr=22500)

  

In [ ]:
gun=[]
for i in range(91):
  try:
    if(i<51):
      shots=os.listdir('gun200/gunaud/'+str(i)+"/")
      for shot in shots:
        gun.append(librosa.load('gun200/gunaud/'+str(i)+"/"+shot))
    else:
      shots=os.listdir('gun200/gunaud/'+str(i)+"./")
      for shot in shots:
        gun.append(librosa.load('gun200/gunaud/'+str(i)+"./"+shot))
    
  except:
    print(i)
    
rifle=[]
for i in range(101):
  try:
    if(i<51):
      shots=os.listdir('gun200/rifleaud/'+str(i)+"/")
      for shot in shots:
        rifle.append(librosa.load('gun200/rifleaud/'+str(i)+"/"+shot))
    else:
      shots=os.listdir('gun200/rifleaud/'+str(i)+"./")
      for shot in shots:
        rifle.append(librosa.load('gun200/rifleaud/'+str(i)+"./"+shot))
  except:
    print(i)   

17
20
40
51
75
80
82
83
88
89
16
27
37
45
47
49
50
67
72
83
86
95


In [ ]:
gun_arr=[]
label=[]#for handgun l is 0
gl1=len(gun)
for i in range(gl1):
  if(gun[i][0].shape[0]==22200):
    fx=np.concatenate((np.array([-1.0]*150),gun[i][0],np.array([-1.0]*150)),axis=0)
    gun_arr.append(feature1(fx))
  elif(gun[i][0].shape[0]==21624):
    fx=np.concatenate((np.array([-1.0]*438),gun[i][0],np.array([-1.0]*438)),axis=0)
    gun_arr.append(feature1(fx))
        
  label.append(0)
rifle_arr=[]
rl1 = len(rifle)
for i in range(rl1):  
  if(rifle[i][0].shape[0]==22200):
    fx=np.concatenate((np.array([-1.0]*150),rifle[i][0],np.array([-1.0]*150)),axis=0)
    rifle_arr.append(feature1(fx))
  elif(rifle[i][0].shape[0]==21624):
    fx=np.concatenate((np.array([-1.0]*438),rifle[i][0],np.array([-1.0]*438)),axis=0)
    rifle_arr.append(feature1(fx))
        
  label.append(1)

In [ ]:
train_mfcc=np.array(gun_arr+rifle_arr)
label= np.array(label)
#tr,ts,ytr,yts=tt(train_mfcc,label,test_size=0.20)
#print(tr.shape,ts.shape,ytr.shape,yts.shape)
t3=train_mfcc.shape
train_mfcc=np.reshape(train_mfcc,(t3[0],t3[1],t3[2],1))
print(train_mfcc.shape,label.shape)

(1339, 428, 44, 1) (1339,)


In [ ]:
drop=0.75
epoch = 100
learning_rate=0.00085
n=5
cv=StratifiedKFold(n_splits=n,shuffle=True)
roctemp2=[]
f1s1=[]
aps1=[]
accs1=[]
aucs1=[]
X=train_mfcc
y=label
count=0
for train1, test1 in cv.split(X,y):
  train,val=X[train1], X[test1]
  ytrain = keras.utils.to_categorical(y[train1])
  yval= keras.utils.to_categorical(y[test1])
  
  #train,val=normalize(train,val)
  #shape lo
  
#   tr_shape=train.shape
#   ts_shape=val.shape
  
#   #reshape
#   val=np.reshape(val,(len(val),-1))
#   train = np.reshape(train,(len(train),-1))
  
# #   pca=PCA()
# #   pca.fit(train2)
# #   train= pca.transform(train2)
# #   test = pca.tranform(test2)

  
#   s = sd()
#   s.fit(train)
#   train = s.transform(train)
#   val = s.transform(val)
  
#   val=np.reshape(val,ts_shape)
#   train = np.reshape(train,tr_shape)
  
  
  
  model = Sequential()
  model.add(Convolution2D(32, (2,2), input_shape=(train_mfcc.shape[1],44,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))



  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  




  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

  model.add(Convolution2D(64, 2, activation='relu',padding='same'))
  #model.add(ZeroPadding2D((1, 1)))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  #model.add(Dropout(0.3))


  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
#   model.add(Dense(512, activation='relu'))
#   model.add(Dense(1024, activation='relu'))
  #model.add(Dropout(0.30))
  #model.add(Dense(1024, activation='relu'))
  model.add(Dropout(drop))
  model.add(Dense(2))
  model.add(Activation('softmax'))
  adam=optimizers.Adam(lr=learning_rate)
  model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
  filepath="k1"+str(count)+".hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  history=model.fit(train,ytrain, epochs=epoch,batch_size=None, verbose=1, validation_data=(val,yval) ,validation_split=None,callbacks=callbacks_list,  shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
  #model.save(dictm.get(mo)+"sound.h5")


  model = Sequential()
  model.add(Convolution2D(32, (2,2), input_shape=(train_mfcc.shape[1],44,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))



  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  

  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))




  model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

  model.add(Convolution2D(64, 2, activation='relu',padding='same'))
  #model.add(ZeroPadding2D((1, 1)))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  #model.add(Dropout(0.3))


  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  #model.add(Dropout(0.30))
#   model.add(Dense(512, activation='relu'))
#   model.add(Dense(1024, activation='relu'))
  model.add(Dropout(drop))
  model.add(Dense(2))
  model.add(Activation('softmax'))
  model.load_weights("k1"+str(count)+".hdf5")
  adam=optimizers.Adam(lr=learning_rate)
  model.compile(optimizer=adam,loss='categorical_crossentropy', metrics=['accuracy'])

  score=model.evaluate(val,yval)
  print(" accuracy", score[1]*100,"%")
  accs1.append(score[1])

  #print(dictm.get(mo)," : ",score[1]*100)
  probs=model.predict_proba(val)
  ys=np.argmax(yval, axis=1)
  auc = roc_auc_score(ys, probs[:,1])
  aucs1.append(auc)
  #yval2=keras.utils.to_categorical(yval)
  fpr, tpr, thresholds = roc_curve(ys, probs[:,1])
  # plt.plot([0.0, 1.0], [0.0, 1.0], linestyle='--')
  # plt.plot(fpr, tpr, marker='.')
  # plt.show()
  f1 = f1_score(ys, model.predict_classes(val))
  f1s1.append(f1)
  ap = average_precision_score(ys, probs[:,1])
  aps1.append(ap)
  roctemp2.append([fpr, tpr, thresholds])
  print("auc, f1, ap " ,auc, f1, ap)
  count+=1
  # conf_mat=confusion_matrix(y, model.predict_classes(valf))
  # print(conf_mat)
  # precision, recall, thresholds = precision_recall_curve(y, probs[:,1])
  # roctemp2.append([precision, recall, thresholds])
  # plt.plot([1.0, 0.0], [0.0, 1.0], linestyle='--')
  # plt.plot(recall, precision, marker='.')
  # plt.show()


Train on 1070 samples, validate on 269 samples
Epoch 1/100
1070/1070 [==============================] - 3s 3ms/step - loss: 0.6937 - acc: 0.4879 - val_loss: 0.6926 - val_acc: 0.5167

Epoch 00001: val_acc improved from -inf to 0.51673, saving model to k10.hdf5
Epoch 2/100
1070/1070 [==============================] - 1s 802us/step - loss: 0.6887 - acc: 0.5168 - val_loss: 0.6587 - val_acc: 0.5167

Epoch 00002: val_acc did not improve from 0.51673
Epoch 3/100
1070/1070 [==============================] - 1s 724us/step - loss: 0.6137 - acc: 0.6860 - val_loss: 0.4764 - val_acc: 0.7918

Epoch 00003: val_acc improved from 0.51673 to 0.79182, saving model to k10.hdf5
Epoch 4/100
1070/1070 [==============================] - 1s 727us/step - loss: 0.4248 - acc: 0.8280 - val_loss: 0.4429 - val_acc: 0.7993

Epoch 00004: val_acc improved from 0.79182 to 0.79926, saving model to k10.hdf5
Epoch 5/100
1070/1070 [==============================] - 1s 726us/step - loss: 0.3643 - acc: 0.8514 - val_loss: 0.23

In [ ]:
print("f1,ap,acc,auc",np.mean(f1s1),np.mean(aps1),np.mean(accs1),np.mean(aucs1))
#f1,ap,acc,auc 0.9643206445997329 0.9891824616238406 0.9634242016337955 0.9888176950078733

f1,ap,acc,auc 0.9680500780087898 0.9862685713383996 0.9663730657672447 0.9883397735416706


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
np.save("./drive/My Drive/1",roctemp2)
# 98.5% acc h 1.1 ki

In [ ]:
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)

tprs=[]
mean_fpr = np.linspace(0, 1, 100)
for i in roctemp:
  tprs.append(np.interp(mean_fpr,i[0],i[1]))
  tprs[-1][0] = 0.0
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0

plt.plot(mean_fpr, mean_tpr, color='b',label='mean_roc',lw=2, alpha=.8)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()